In [ ]:
from datetime import datetime

In [ ]:
fecha = datetime.today()#.strftime("%d-%m-%y")
print(fecha.strftime("%d-%m-%y"))
#print(f"{fecha.strftime('%d-%m-%y')}")

In [ ]:
from util.datajson import DataJson
#form datetime import datetime

In [ ]:
settings = DataJson("settings", dict())
time_schedule = settings.add_and_get_dict_value_if_not_exist("schedule", {"no_check_in" : ["08:20"]})
time_schedule = time_schedule[0]["no_check_in"]

#print(time_schedule)

time_schedule = [datetime.strptime(time, "%H:%M").time() for time in time_schedule]

#print(time_schedule)

#at = datetime.now()

#at = at.replace(second=0, microsecond=0).time()

print(f"{datetime.now().remplace(second=0, microsecond=0).time().strftime('%H:%M')}")

AGREGAR FAKE DATA A LA BASE DE DATOS.

In [ ]:

import sqlite3

def obtener_estudiantes():
    conn = sqlite3.connect('miguardian.db')
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM student")
    estudiantes = cursor.fetchall()
    conn.close()
    return estudiantes

def insertar_asistencias(estudiantes, fecha, entrada, salida):
    conn = sqlite3.connect('miguardian.db')
    cursor = conn.cursor()
    for estudiante in estudiantes:
        cursor.execute("INSERT INTO attendance (date, entry_time, exit_time, student_id) VALUES (?, ?, ?, ?)", (fecha, entrada, salida, estudiante[0]))
    conn.commit()
    conn.close()


In [ ]:
insertar_asistencias(obtener_estudiantes(), '2024-05-26', '08:00:00', '15:00:00')

In [ ]:
import logging
import json

In [ ]:
class JsonFormatter(logging.Formatter):
    def format(self, record):
        log_record = {
            "time": self.formatTime(record, self.datefmt),
            "name": record.name,
            "level": record.levelname,
            "message": record.getMessage()
        }
        if record.exc_info:
            log_record['exc_info'] = self.formatException(record.exc_info)
        return json.dumps(log_record)

In [ ]:
logging.basicConfig(level=logging.DEBUG, filename="app.log", filemode="w")
logger = logging.getLogger()
handler = logging.FileHandler("app.log")
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)

In [ ]:
import database.miguardiandb as db
from datetime import datetime

fecha_hoy = datetime.today().strftime('%Y-%m-%d')
df = db.get_noCheckIn_student()
tamaño_df = len(df) 
list_alumnos_nombres = []
contador = 1
alumnos_nombres = ""
#sublistas_alumnos = [df.iloc[i:i + 30] for i in range(0, len(df), 30)]##sublistas_alumnos = [list_student[i:i + 30] for i in range(0, len(list_student), 30)]

for index, alumno in df.iterrows():
    #generamos el mensaje de cada alumno tomando los datos de cada iteracion del data frame
    alumnos_nombres += f"{index + 1}. {alumno['nombre']} {alumno['apellidos']}-{alumno['grado']}º{alumno['grupo']}\n"
        
    #El index se va a dividir entre treinta cuando el residuo 
    # sean un valor 0 va a guardar otra ronda de mensajes
    contador = (index + 1) % 30
    
    if contador == 0 or index == tamaño_df - 1:
        mensaje = f"Alumnos que no registraron entrada el {fecha_hoy}: \n" + "".join(alumnos_nombres)
        list_alumnos_nombres.append(mensaje)
    

for mensaje in list_alumnos_nombres:
    print(mensaje)
#for indice, row in alumnos_sin_asistencia.iterrows():
#    print(f"{row['nombre']} {row['apellidos']}")
    # comment: 
# end for

In [ ]:
import eventNotification as en

en.notification_no_check_in()

In [33]:
from util.notification import TelegramNotifier
import pandas as pd
import urllib.parse
import time

In [38]:

# Ruta del archivo CSV local
csv_path = "/Users/ikioriy/Downloads/lazaro-cardenas/notificar_lazaro_cardenas.csv"
id_column = "id_telegram"

# Cargar archivo CSV
df = pd.read_csv(csv_path)

# Inicializar clase para enviar mensajes
telegram_notification = TelegramNotifier()

# Configuración de WhatsApp de asistencia
numero = "523312100172"
mensaje_whatsapp = "Solicito asistencia para llenar el formulario de actualización de datos en miEscuela"
mensaje_codificado = urllib.parse.quote(mensaje_whatsapp)
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"

# Ruta de la imagen a enviar
imagen_path = "/Users/ikioriy/MEGA/nuevo_diseño.jpg"

# Enviar mensajes
enviados = []

for index, row in df.iterrows():
    chat_id = row.get(id_column)
    if pd.isna(chat_id):
        continue

    try:
        chat_id_str = str(int(chat_id))  # Asegura string limpio

        mensaje_completo = f"""
*¡Atención padres de familia\!*
        
👉 Lee atentamente, *llenen el formulario con los datos actualizados del alumno y del tutor*\. 🪪👨‍👩‍👧‍👦 

📌 Asegúrate de tener listos los siguientes elementos:
1\. Cuenta de correo electrónico vigente\.
2\. Contraseña segura que no vayas a olvidar \(mínimo 8 caracteres, una mayúscula y un número, ejemplo: *Sotelo99*\)\.
3\. CURP del alumno\.
4\. Fotografías del alumno y del tutor\.
5\. Haber instalado la aplicación Telegram\.
6\. 🤖 *Tu ID de Telegram registrado es:* \`{chat_id_str}\`  
\(Guárdalo lo vas a necesitar más adelante\)
7\. Leer con atención todas las instrucciones del formulario\.

🔐 Registrate en el portal aquí:
https://miescuela\.net/register

📝 Llena el formulario aquí:
https://miescuela\.net/formulario

✏️ Después puedes consultar o actualizar tu informarción aquí:
https://miescuela\.net/mis\-datos

📺 Video tutorial paso a paso:
https://youtu\.be/3myT3epxwBo

🗓️ *Fecha límite:* _22 de Julio de 2025_  
Después de esa fecha, las credenciales se imprimirán con los datos actuales registrados en el sistema\.

🙏🏻 Gracias por tu apoyo y compromiso\.

*Escuela primaria Lázaro Cárdenas del Río*

📲 ¿Necesitas ayuda?  
[Solicitar asistencia por WhatsApp]({whatsapp_link})
 """

        print(f"Enviando a chat_id: {chat_id_str}")
        telegram_notification.send_message("1514049372"  , mensaje_completo)#chat_id, mensaje_completo)
        time.sleep(2)
        enviados.append(chat_id)#chat_id)

    except Exception as e:
        print(f"❌ Error al enviar a {chat_id}: {e}")

print(f"✅ Mensajes enviados: {len(enviados)}")


❌ Error al enviar a  : invalid literal for int() with base 10: ' '
❌ Error al enviar a  : invalid literal for int() with base 10: ' '
Enviando a chat_id: 1323264228
{'ok': True, 'result': {'message_id': 842381, 'from': {'id': 6061799324, 'is_bot': True, 'first_name': 'miEscuela', 'username': 'mi_escuela_bot'}, 'chat': {'id': 1514049372, 'first_name': 'Sergio Mares', 'username': 'Seryi_0', 'type': 'private'}, 'date': 1752178188, 'text': '¡Atención padres de familia!\n        \n👉 Lee atentamente, llenen el formulario con los datos actualizados del alumno y del tutor. 🪪👨\u200d👩\u200d👧\u200d👦 \n\n📌 Asegúrate de tener listos los siguientes elementos:\n1. Cuenta de correo electrónico vigente.\n2. Contraseña segura que no vayas a olvidar (mínimo 8 caracteres, una mayúscula y un número, ejemplo: Sotelo99).\n3. CURP del alumno.\n4. Fotografías del alumno y del tutor.\n5. Haber instalado la aplicación Telegram.\n6. 🤖 Tu ID de Telegram registrado es: `1323264228`  \n(Guárdalo lo vas a necesitar m

KeyboardInterrupt: 

In [ ]:
for index, row in df.iterrows():
    chat_id = row.get(id_column)
    if pd.isna(chat_id):
        continue

    try:
        chat_id_str = str(int(chat_id))  # Asegura que sea texto limpio

        # Mensaje completo con ID personalizado
        mensaje_completo = f"""
👉 Para ello, es necesario que *llenen un formulario con los datos actualizados del alumno y del tutor*.

📌 Asegúrate de tener listos los siguientes elementos:
1. Cuenta de correo electrónico vigente.
2. Contraseña segura que no vayas a olvidar (mínimo 8 caracteres, una mayúscula y un número, ejemplo: *Sotelo99*).
3. CURP del alumno.
4. Fotografías del alumno y del tutor.
5. Haber instalado la aplicación Telegram.
6. Leer con atención todas las instrucciones del formulario.

📝 Llena el formulario aquí:  
https://miescuela.net/formulario/?clave_instituto=14DPR0000Y

📺 Video tutorial paso a paso:  
https://youtu.be/3myT3epxwBo

🤖 *Tu ID de Telegram registrado es:* `{chat_id_str}`  
(Guárdalo por si lo necesitas más adelante)

🗓️ *Fecha límite:* _19 de agosto de 2024_  
Después de esa fecha, las credenciales se imprimirán con los datos actuales registrados en el sistema.

🙏🏻 Gracias por tu apoyo y compromiso.

*Escuela primaria Lázaro Cárdenas del Río*

📲 ¿Necesitas ayuda?  
[Solicitar asistencia por WhatsApp]({whatsapp_link})
"""

        print(f"Enviando mensaje a chat_id: {chat_id_str}")
        telegram_notification.send_message(chat_id, mensaje_con_imagen, imagen_path)
        time.sleep(1)
        telegram_notification.send_message(chat_id, mensaje_completo)
        time.sleep(2)
        enviados.append(chat_id)

    except Exception as e:
        print(f"❌ Error al enviar a {chat_id}: {e}")


In [ ]:

# Ruta del CSV local
csv_path = "/Users/ikioriy/Downloads/lazaro-cardenas/notificar_lazaro_cardenas.csv"
id_column = "id_telegram"  # Asegúrate que esta columna exista

# Cargar archivo CSV
df = pd.read_csv(csv_path)

# Inicializar clase para enviar mensajes
telegram_notification = TelegramNotifier()

# Configuración de WhatsApp de asistencia
numero = "523312100172"
mensaje_whatsapp = "Solicito asistencia para llenar el formulario de actualización de datos en miEscuela"
mensaje_codificado = urllib.parse.quote(mensaje_whatsapp)
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"


# Mensaje completo con instrucciones y enlaces
mensaje_completo = f"""

*¡Atención padres de familia\!*

Sigue los siguientes pasos para cargar la informacion del alumno y tutores desde el nuevo formulario en miescuela.net*\. 🪪👨‍👩‍👧‍👦

👉 Para ello, es necesario que *llenen un formulario con los datos actualizados del alumno y del tutor*\.

📌 Asegúrate de tener listos los siguientes elementos\:
1\. Cuenta de correo electrónico vigente\.
2\. Contraseña segura que no vayas a olvidar (mínimo 8 caracteres, una mayúscula y un número, ejemplo\: *Sotelo99*)\.
3\. CURP del alumno\.
4\. Fotografías del alumno y del tutor\.
5\. Haber instalado la aplicación Telegram\.
6\. Leer con atención todas las instrucciones del formulario\.

📝 Llena el formulario aquí\:  
https://miescuela.net/login/

📺 Video tutorial paso a paso\:  
https://youtu.be/3myT3epxwBo

🤖 Además, para obtener tu ID de Telegram (necesario para notificar entradas y salidas del alumno), envía el siguiente comando al bot\:  
👉 https://t.me/mi_escuela_bot

🗓️ *Fecha límite:* 22 de julio de 2025  
Después de esa fecha, las credenciales se imprimirán con los datos actuales registrados en el sistema\.

🙏🏻 Gracias por tu apoyo y compromiso\.

*Escuela primaria Lázaro Cárdenas del Río*

📲 ¿Necesitas ayuda?  
[Solicitar asistencia por WhatsApp]({whatsapp_link})
"""

# Ruta de la imagen a enviar (puedes ajustar esta ruta)
#imagen_path = "/Users/ikioriy/MEGA/nuevo_diseño.jpg"

# Enviar mensajes
enviados = []
'''
for index, row in df.iterrows():
    chat_id = row.get(id_column)
    if pd.isna(chat_id):
        continue

    try:
        print(f"Enviando a chat_id: {chat_id}")
        #telegram_notification.send_message(chat_id, mensaje_con_imagen)
        #time.sleep(1)
        telegram_notification.send_message(chat_id, mensaje_completo)
        enviados.append(chat_id)
        time.sleep(2)
    except Exception as e:
        print(f"❌ Error al enviar a {chat_id}: {e}")

'''

try:
    print(f"Enviando a chat_id: 1323264228")
    #telegram_notification.send_message(chat_id, mensaje_con_imagen)
    #time.sleep(1)
    telegram_notification.send_message(1323264228, mensaje_completo)
    enviados.append(1323264228)
    time.sleep(2)
except Exception as e:
    print(f"❌ Error al enviar a {1323264228}: {e}")

# Guardar log de enviados (opcional)
log_path = "/Users/ikioriy/Downloads/lazaro-cardenas/log_mensajes_enviados.csv"
pd.DataFrame(enviados, columns=["chat_id"]).to_csv(log_path, index=False)
print(f"✅ Mensajes enviados: {len(enviados)}")
print(f"📄 Log guardado en: {log_path}")


In [ ]:
import pandas as pd

# Ruta al archivo CSV
csv_path = '/Users/ikioriy/Desktop/secundaria_153/secundaria_153_para_imprimir.csv'#'/Users/ikioriy/Downloads/las_americas_notificacion1.csv'

# Leer el archivo CSV
df = pd.read_csv(csv_path, dtype={'chat_id1': str})

# Acceder a la columna 'chat_id1'
filtered_df = df[df['chat_id1'].notna() & (df['chat_id1'] != '') & (df['chat_id1'] != '0')]

#print(filtered_df)

# Contar el número de registros en la columna 'chat_id1' después del filtrado
count = filtered_df.count()

# Mostrar el conteo de registros
print(f"\nEl número de chat_id1 en el archivo es: {count}")


In [ ]:
telegram_notification = TelegramNotifier()

mensaje = f"""
*Estimados padres de familia,*

Nos complace informarles que hemos recibido la confirmación de sus datos a través del bot de Telegram miEscuela y que el pago correspondiente ha sido registrado exitosamente\.

Por lo tanto, la credencial de su hijo\(a\) está lista para ser entregada\. Les agradecemos por su colaboración y cumplimiento en el proceso\.

Si tienen alguna pregunta adicional, no duden en ponerse en contacto con nosotros\.

Atentamente,
*Escula Secundaria Técnica 153\.*
"""
notificados = []
no_notificados = []
counter = 0

for index, row in filtered_df.iterrows():
    chat_id = row['chat_id1']
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    
    response = telegram_notification.send_message(chat_id, mensaje)
    if response.get("ok"):
        notificados.append(row)
    else:
        no_notificados.append(row)
    time.sleep(2)
    
df_notificados = pd.DataFrame(notificados)
df_no_notificados = pd.DataFrame(no_notificados)

# Guardar los DataFrames en archivos CSV
df_notificados.to_csv('/Users/ikioriy/Desktop/secundaria_153/secundaria_153_notificados.csv', index=False)
df_no_notificados.to_csv('/Users/ikioriy/Desktop/secundaria_153/secundaria_153_no_notificado.csv', index=False)

print(f"Se enviaron {counter} mensajes en total.")

#telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
#telegram_notification.send_message(1323264228, mensaje_con_video)


In [ ]:
telegram_notification = TelegramNotifier()

message = f"El 👨‍🎓 alumno registro su entrada a las ⏰ 1:46 pm 25 febrero 2025"

response = telegram_notification.send_message(6382856411, message)

print(response)

### ENVIO DE MENSAJES MASIVO BOT TELEGRAM LAZARO CARDENAS

In [ ]:
telegram_notification = TelegramNotifier()

# Codificar el mensaje de WhatsApp
mensaje = "Solicito asistencia para actualizar información en el bot miEscuela de Telegram"
mensaje_codificado = urllib.parse.quote(mensaje)

# Crear el enlace de WhatsApp
numero = "523312100172"
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"

mensaje_con_imagen = f"""
*Estimados padres de familia,*
*Nuevo diseño de credencial con los datos de los tutores impreso*
"""
mensaje_con_video = f"""
Es importante que actualicen y confirmen sus datos ✅ en nuestro bot de Telegram 🤖 para asegurarse de que su credencial se imprima con la información correcta, ademas de actualizar la inforamción de los tutotres ya que vendran impresos en el nuevo diseño de la credencial\.

Recuerden que el día 🗓️ _*19 de Agosto del 2024*_ es la *fecha límite* 🙀 para realizar los cambios, específicamente algún cambio en los datos de los Alumnos 🧑‍🎓 como el tipo de sangre y actualizar los datos de los tutores que vendran impresos en el nuevo diseño de la credencial\. A partir de esa fecha limite, las credenciales se imprimirán con los datos existentes en nuestro sistema\.

Por favor, dediquen unos minutos para verificar y actualizar sus datos si es necesario\. De esta manera, evitaremos errores y aseguraremos que su credencial esté lista para el nuevo ciclo escolar\.👍

Se adjunta video tutorial 🎥 para realizar la confirmación y actualizacion de los datos en el bot de Telegram: [Video tutorial](https://youtu.be/u1edkTBzjrM)\.

🙏🏻 Gracias por su atención y cooperación\.

Atentamente,  
Escuela primaria *Lázaro Cárdenas del Río\.*

Para recibir asistencia, haga clic en el siguiente enlace: [Solicitar asistencia en WhatsApp]({whatsapp_link})
"""

imagen_path = "/Users/ikioriy/MEGA/nuevo_diseño.jpg"

counter = 0

'''for chat_id in filtered_df['chat_id1']:
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    telegram_notification.send_message(chat_id, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
    time.sleep(1)  # import time
    telegram_notification.send_message(chat_id, mensaje_con_video)
    time.sleep(3)
    
print(f"Se enviaron {counter} mensajes en total.")'''

telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
telegram_notification.send_message(1323264228, mensaje_con_video)


### NOTIFICACION 2 LAZARO CARDENAS

In [ ]:
telegram_notification = TelegramNotifier()

# Codificar el mensaje de WhatsApp
mensaje_whatsapp = "Solicito asistencia para actualizar información en el bot miEscuela de Telegram"
mensaje_codificado = urllib.parse.quote(mensaje_whatsapp)

# Crear el enlace de WhatsApp
numero = "523312100172"
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"

mensaje = f"""
*Estimados padres de familia,*
*Nuevo diseño de credencial con los datos de los tutores impreso*

_*Recuerda que tienes que confirmar datos*_ 🙀 en nuestro bot en telegram, si ya revistate la información y es correcta hay que confirmar datos, es parte importante del proceso\.

Por favor, dediquen unos minutos para verificar y actualizar sus datos si es necesario\. De esta manera, evitaremos errores y aseguraremos que su credencial esté lista para el nuevo ciclo escolar\.👍

Se adjunta video tutorial 🎥 para realizar la actualizacion y confirmación de los datos en el bot de Telegram: [Video tutorial](https://youtu.be/u1edkTBzjrM)\.

🙏🏻 Gracias por su atención y cooperación\.

Atentamente,  
Escuela primaria *Lázaro Cárdenas del Río\.*

Si necesitas asistencia, haga clic en el siguiente enlace: [Solicitar asistencia en WhatsApp]({whatsapp_link})
"""

notificados = []
no_notificados = []
counter = 0

for index, row in filtered_df.iterrows():
    chat_id = row['chat_id1']
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    
    response = telegram_notification.send_message(chat_id, mensaje)
    if response.get("ok"):
        notificados.append(row)
    else:
        no_notificados.append(row)
    time.sleep(2)
    
df_notificados = pd.DataFrame(notificados)
df_no_notificados = pd.DataFrame(no_notificados)

# Guardar los DataFrames en archivos CSV
df_notificados.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas_2_notificados.csv', index=False)
df_no_notificados.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas_2_no_notificado.csv', index=False)

print(f"Se enviaron {counter} mensajes en total.")


#telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
#telegram_notification.send_message(1323264228, mensaje_con_video)


### NOTIFICACION LAZARO CARDENAS 3

In [ ]:
telegram_notification = TelegramNotifier()

# Codificar el mensaje de WhatsApp
mensaje_whatsapp = "Solicito asistencia para actualizar información en el bot miEscuela de Telegram"
mensaje_codificado = urllib.parse.quote(mensaje_whatsapp)

# Crear el enlace de WhatsApp
numero = "523312100172"
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"

mensaje = f"""
*Estimados padres de familia,*
*DATOS NO CONFIRMADOS* 🥹

_*Recuerda que tienes que confirmar datos*_ ❌, si estas recibiendo este mensaje es por que aun no haz confirmado tus datos en el bot de telegram 🙀\. 

Se imprimiran las credenciales 🪪 con los datos existentes en la base de datos actual, de imprimirse con un error y requerir corrección generar un costo de reimpresión 💸\.

🗓️ _*Fecha limite de confirmacion 22 de agosto del 2024*_ 🗓️\.

Por favor, dediquen unos minutos para verificar y actualizar sus datos si es necesario\. De esta manera, evitaremos errores y aseguraremos que su credencial esté lista para el nuevo ciclo escolar\.👍

Se adjunta video tutorial 🎥 para realizar la actualizacion y confirmación de los datos en el bot de Telegram: [Video tutorial](https://youtu.be/u1edkTBzjrM)\.

🙏🏻 Gracias por su atención y cooperación\.

Atentamente,  
Escuela primaria *Lázaro Cárdenas del Río\.*

Si necesitas asistencia, haga clic en el siguiente enlace: [Solicitar asistencia en WhatsApp]({whatsapp_link})
"""

notificados = []
no_notificados = []
counter = 0

for index, row in filtered_df.iterrows():
    chat_id = row['chat_id1']
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    
    response = telegram_notification.send_message(chat_id, mensaje)
    if response.get("ok"):
        notificados.append(row)
    else:
        no_notificados.append(row)
    time.sleep(2)
    
df_notificados = pd.DataFrame(notificados)
df_no_notificados = pd.DataFrame(no_notificados)

# Guardar los DataFrames en archivos CSV
df_notificados.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas_3_notificados.csv', index=False)
df_no_notificados.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas_3_no_notificado.csv', index=False)

print(f"Se enviaron {counter} mensajes en total.")


#telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
#telegram_notification.send_message(1323264228, mensaje_con_video)


### MENSAJE LAZARO CARDENAS 4

In [ ]:
telegram_notification = TelegramNotifier()

mensaje = f"""
*Estimados padres de familia,*
*DATOS NO CONFIRMADOS* 🥹

Tu credencial sera impresa con los datos existentes en nuestar base de datos actual 🪪, de imprimirse con un error y requerir corrección generar un costo de reimpresión 💸\.

🗓️ _*ENTREGA PROGRAMADA PARA EL DIA 26 DE AGOSTO*_ 🗓️\.

🙏🏻 Gracias por su atención y cooperación\.

Atentamente,  
Escuela primaria *Lázaro Cárdenas del Río\.*

"""

notificados = []
no_notificados = []
counter = 0

for index, row in filtered_df.iterrows():
    chat_id = row['chat_id1']
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    
    response = telegram_notification.send_message(chat_id, mensaje)
    if response.get("ok"):
        notificados.append(row)
    else:
        no_notificados.append(row)
    time.sleep(2)
    
df_notificados = pd.DataFrame(notificados)
df_no_notificados = pd.DataFrame(no_notificados)

# Guardar los DataFrames en archivos CSV
df_notificados.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_4_notificados.csv', index=False)
df_no_notificados.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_4_no_notificado.csv', index=False)

print(f"Se enviaron {counter} mensajes en total.")


#telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
#telegram_notification.send_message(1323264228, mensaje_con_video)


### NOTIFICACION LAS AMERICAS DE GUADALAJARA

In [ ]:
telegram_notification = TelegramNotifier()

# Codificar el mensaje de WhatsApp
mensaje_whatsapp = "Solicito asistencia para actualizar información en el bot miEscuela de Telegram"
mensaje_codificado = urllib.parse.quote(mensaje_whatsapp)

# Crear el enlace de WhatsApp
numero = "523312100172"
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"

mensaje = f"""
*Estimados padres de familia,*
*DATOS NO CONFIRMADOS* 🥹

_*Recuerda que tienes que confirmar datos*_ ❌, si estas recibiendo este mensaje es por que aun no haz confirmado tus datos en el bot de telegram 🙀\. 

Se imprimiran las credenciales 🪪 con los datos existentes en la base de datos actual, de imprimirse con un error y requerir corrección generar un costo de reimpresión 💸\.

🗓️ _*Fecha limite de confirmacion 27 de agosto del 2024*_ 🗓️\.

Por favor, dediquen unos minutos para verificar y actualizar sus datos si es necesario\. De esta manera, evitaremos errores y aseguraremos que su credencial esté lista para el nuevo ciclo escolar\.👍

Se adjunta video tutorial 🎥 para realizar la actualizacion y confirmación de los datos en el bot de Telegram: [Video tutorial](https://youtu.be/u1edkTBzjrM)\.

🙏🏻 Gracias por su atención y cooperación\.

Atentamente,  
Colegio Las Americas de Guadalajara\.*

Si necesitas asistencia, haga clic en el siguiente enlace: [Solicitar asistencia en WhatsApp]({whatsapp_link})
"""

notificados = []
no_notificados = []
counter = 0

for index, row in filtered_df.iterrows():
    chat_id = row['chat_id1']
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    
    response = telegram_notification.send_message(chat_id, mensaje)
    if response.get("ok"):
        notificados.append(row)
    else:
        no_notificados.append(row)
    time.sleep(2)
    
df_notificados = pd.DataFrame(notificados)
df_no_notificados = pd.DataFrame(no_notificados)

# Guardar los DataFrames en archivos CSV
df_notificados.to_csv('/Users/ikioriy/Desktop/las_americas_notificados.csv', index=False)
df_no_notificados.to_csv('/Users/ikioriy/Desktop/las_americas_no_notificado.csv', index=False)

print(f"Se enviaron {counter} mensajes en total.")

#telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
#telegram_notification.send_message(1323264228, mensaje_con_video)


### NOTIFICACION 2 LAS AMERICAS 

In [ ]:
telegram_notification = TelegramNotifier()

# Codificar el mensaje de WhatsApp
mensaje_whatsapp = "Solicito asistencia para actualizar información en el bot miEscuela de Telegram"
mensaje_codificado = urllib.parse.quote(mensaje_whatsapp)

# Crear el enlace de WhatsApp
numero = "523312100172"
whatsapp_link = f"https://wa.me/{numero}?text={mensaje_codificado}"

mensaje = f"""
*Estimados padres de familia,*

Es importante que actualicen y confirmen sus datos ✅ en nuestro bot de Telegram 🤖 para asegurarse de que su credencial se imprima con la información correcta, ademas de actualizar la información\.

Recuerden que el día 🗓️ _*24 de Agosto del 2024*_ es la *fecha límite* 🙀 para realizar los cambios, específicamente algún cambio en los datos de los Alumnos 🧑‍🎓 como la fotografia y actualizar los datos de los tutores como el número de telélefono\. A partir de esa fecha limite, las credenciales se imprimirán con los datos existentes en nuestro sistema\.

Por favor, dediquen unos minutos para verificar y actualizar sus datos si es necesario\. De esta manera, evitaremos errores y aseguraremos que su credencial esté lista para el nuevo ciclo escolar\.👍

Recurda que las impresiones se realizaran tambien cuando se haya cubierto el *pago del seguro escolar* 🏥\.

Se adjunta video tutorial 🎥 para realizar la actualizacion y confirmación de los datos en el bot de Telegram: [Video tutorial](https://youtu.be/jj20xbB3jyg)\.

🙏🏻 Gracias por su atención y cooperación\.

Atentamente,  
*Colegio Las Americas de Guadalajara\.*

Para recibir asistencia, haga clic en el siguiente enlace: [Solicitar asistencia en WhatsApp]({whatsapp_link})
"""
notificados = []
no_notificados = []
counter = 0

for index, row in filtered_df.iterrows():
    chat_id = row['chat_id1']
    counter += 1
    print(f"Enviando mensaje {counter} a chat_id: {chat_id}")
    
    response = telegram_notification.send_message(chat_id, mensaje)
    if response.get("ok"):
        notificados.append(row)
    else:
        no_notificados.append(row)
    time.sleep(2)
    
df_notificados = pd.DataFrame(notificados)
df_no_notificados = pd.DataFrame(no_notificados)

# Guardar los DataFrames en archivos CSV
df_notificados.to_csv('/Users/ikioriy/Desktop/las_americas_2_notificados.csv', index=False)
df_no_notificados.to_csv('/Users/ikioriy/Desktop/las_americas_2_no_notificado.csv', index=False)

print(f"Se enviaron {counter} mensajes en total.")

#telegram_notification.send_message(1323264228, mensaje_con_imagen, imagen_path)# 1514049372, mensaje  )
#telegram_notification.send_message(1323264228, mensaje_con_video)


### SABER SI EXISTE UN ESTUDIANTE EN LA BASE DE DATOS

In [ ]:
import sqlite3

def get_chat_id1_by_student_name(db_path, student_name):
    # Conectar a la base de datos
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Consulta SQL para obtener el chat_id1 (idTelegram) del tutor basado en el nombre del alumno
    query = """
    SELECT Tutor.idTelegram
    FROM Persons
    JOIN Student ON Persons.id = Student.personid
    JOIN Tutor ON Student.tutorid_tutor = Tutor.id_tutor
    WHERE Persons.nombre = ?
    """

    # Ejecutar la consulta y obtener el resultado
    cursor.execute(query, (student_name,))
    result = cursor.fetchone()

    # Cerrar la conexión a la base de datos
    conn.close()

    # Verificar si se encontró un resultado
    if result:
        return result[0]  # Retorna chat_id1 (idTelegram)
    else:
        return None  # Retorna None si no se encontró al alumno

# Ejemplo de uso
db_path = '/Users/ikioriy/Desktop/Nuevos_Registros.db'
student_name = 'Sofia Montserrat'
chat_id1 = get_chat_id1_by_student_name(db_path, student_name)

if chat_id1:
    print(f"El chat_id1 del tutor de {student_name} es: {chat_id1}")
else:
    print(f"No se encontró ningún tutor para el alumno {student_name}.")
#6819548860

### NORMARLIZAR ARCHIVO CSV DE LA RESPUESTA DEL BOT

In [ ]:
import pandas as pd

# Leer el archivo CSV
csv_path = '/Users/ikioriy/Desktop/lazaro_cardenas.csv'
df = pd.read_csv(csv_path)

# Eliminar filas vacías en la columna 'RESPUESTA JSON'
df = df[df['Respuesta JSON'].notna()]

# Eliminar filas donde 'RESPUESTA JSON' sea 'FALSE' o 'False'
df = df[~df['Respuesta JSON'].isin(['FALSE', 'False'])]

# Eliminar filas con IDs duplicados, conservando solo la primera aparición
df = df.drop_duplicates(subset='ID', keep='first')

# Guardar el DataFrame limpio en un nuevo archivo CSV o sobrescribir el existente
df.to_csv('/Users/ikioriy/Desktop/lazaro_cardenas_clean.csv', index=False)

print("Filas vacías, filas con 'FALSE' o 'False' eliminadas y IDs duplicados eliminados.")


### FILTRAR REGISTROS QUE NO LES LLEGO MENSAJE DEL BOT

In [ ]:
import pandas as pd

# Rutas a los archivos CSV
original_csv_path = '/Users/ikioriy/Downloads/lazaro_cardenas.csv'
clean_csv_path = '/Users/ikioriy/Desktop/lazaro_cardenas_clean.csv'

# Cargar los archivos CSV en DataFrames
df_original = pd.read_csv(original_csv_path)
df_clean = pd.read_csv(clean_csv_path)

# Filtrar los registros que están en el archivo original y coinciden con el archivo limpio
filtered_df = df_original[df_original['chat_id1'].isin(df_clean['ID'])]

# Eliminar filas con IDs duplicados en 'chat_id1', conservando solo la primera aparición
filtered_df = filtered_df.drop_duplicates(subset='chat_id1', keep='first')

# Guardar los registros filtrados y sin duplicados en un nuevo archivo CSV
filtered_csv_path = '/Users/ikioriy/Desktop/filter_message_error.csv'
filtered_df.to_csv(filtered_csv_path, index=False)

print(f"Filtro completado y duplicados eliminados. Los registros se han guardado en '{filtered_csv_path}'.")


### FILTRAR SOLO REGISTROS QUE SI LES LLEGO LA NOTIFICACION DEL BOT

In [ ]:
import pandas as pd

# Rutas a los archivos CSV
original_csv_path = '/Users/ikioriy/Downloads/lazaro_cardenas.csv'
clean_csv_path = '/Users/ikioriy/Desktop/lazaro_cardenas_clean.csv'

# Cargar los archivos CSV en DataFrames
df_original = pd.read_csv(original_csv_path)
df_clean = pd.read_csv(clean_csv_path)

# Filtrar los registros que están en el archivo original pero NO en el archivo limpio
filtered_df = df_original[~df_original['chat_id1'].isin(df_clean['chat_id1'])]

# Guardar los registros filtrados en un nuevo archivo CSV
filtered_csv_path = '/Users/ikioriy/Desktop/filtered_not_in_clean.csv'
filtered_df.to_csv(filtered_csv_path, index=False)

print(f"Filtro completado. Los registros no presentes en el archivo limpio se han guardado en '{filtered_csv_path}'.")


### REVISAR DE LOS QUE YA SE LES MANDO EL MENSAJE QUIEN YA CONFIRMO DATOS

In [ ]:
import pandas as pd
import json

clave_instituto = '14DPR2576Y'#'14PPR1613J'#'14DPR2576Y'
# Rutas a los archivos
json_path = '/Users/ikioriy/MEGA/Proyectos/telegramregistroLazaro-master/tzolkin-post/bin/Release/net6.0-windows/confirmations.json'
csv_path = '/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_exportacion.csv'#'/Users/ikioriy/Desktop/lazaro_cardenas_3_notificados.csv'

# Archivos de salida
output_csv_confirmados = '/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_confirmados_imprimir_hoy.csv'
output_csv_no_confirmados = '/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_no_confirmados_hoy.csv'

# Cargar el archivo JSON de confirmaciones
with open(json_path, 'r') as file:
    confirmations = json.load(file)

# Convertir la lista de confirmaciones a un DataFrame
df_confirmations = pd.DataFrame(confirmations)

# Asegurarse de que `UserId` en `df_confirmations` sea tratado como string
df_confirmations['UserId'] = df_confirmations['UserId'].astype(str)

# Filtrar el DataFrame de confirmaciones para la claveInstituto específica
df_confirmados_instituto = df_confirmations[df_confirmations['claveInstituto'] == clave_instituto]

# Cargar el archivo CSV de notificados asegurándose de que `chat_id1` sea string
df_notificados = pd.read_csv(csv_path, dtype={'chat_id1': str})

# Obtener los registros que ya confirmaron comparando `UserId` del JSON con `chat_id1` del CSV
df_confirmados_final = df_notificados[df_notificados['chat_id1'].isin(df_confirmados_instituto['UserId'])]

# Obtener los registros que no han confirmado
df_no_confirmados_final = df_notificados[~df_notificados['chat_id1'].isin(df_confirmados_instituto['UserId'])]

# Guardar los DataFrames en archivos CSV
df_confirmados_final.to_csv(output_csv_confirmados, index=False)
df_no_confirmados_final.to_csv(output_csv_no_confirmados, index=False)

print(f"Archivos CSV generados:")
print(f" - {output_csv_confirmados}")
print(f" - {output_csv_no_confirmados}")



### COINCIDENCIAS DEL ARCHIVO DEL DIRECTOR CON EL ARCHIVO DE LA EXPORTACION DEL BOT

In [ ]:
import pandas as pd
import unicodedata

# Función para normalizar texto
def normalize_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar acentos
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # Quitar espacios al principio y al final
    return text.strip()

# Rutas a los archivos
base_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/idolina_listas.csv'
secondary_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N.csv'
output_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/idolina_join.csv'
no_match_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/sin_coincidencias_TV.csv'

# Cargar los archivos CSV, especificando los tipos de datos
df_base = pd.read_csv(base_csv_path, dtype={'grado': int, 'chat_id1': str})
df_secondary = pd.read_csv(secondary_csv_path, dtype={'grado': int, 'chat_id1': str})

# Concatenar las columnas 'apellido' y 'nombre' en df_secondary para que coincidan con la columna 'nombre' en df_base
df_secondary['nombre_completo'] = df_secondary['apellidos'] + ' ' + df_secondary['nombre']

# Normalizar las columnas `nombre` en ambos DataFrames
df_base['nombre_completo'] = df_base['nombre_completo'].apply(normalize_text)
df_secondary['nombre_completo'] = df_secondary['nombre_completo'].apply(normalize_text)
#df_base['apellido'] = df_base['apellido'].apply(normalize_text)
#df_secondary['apellido'] = df_secondary['apellido'].apply(normalize_text)

# Realizar el merge utilizando 'nombre_completo' en df_secondary y 'nombre' en df_base
merged_df = pd.merge(df_base, df_secondary, left_on='nombre_completo', right_on='nombre_completo', how='left', indicator=True)

# Realizar la unión de ambos DataFrames solo por la columna `nombre`
#merged_df = pd.merge(df_base, df_secondary, on='nombre', how='left', indicator=True)

# Filtrar las coincidencias encontradas
df_matched = merged_df[merged_df['_merge'] == 'both'].drop(columns=['_merge'])

df_matched = df_matched.drop_duplicates(subset=['nombre_x', 'apellidos_x'], keep='first')

# Filtrar las coincidencias no encontradas
df_no_match = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# Guardar los DataFrames en archivos CSV
df_matched.to_csv(output_csv_path, index=False)
df_no_match.to_csv(no_match_csv_path, index=False)

print(f"Archivo CSV actualizado guardado en: {output_csv_path}")
print(f"Archivo sin coincidencias guardado en: {no_match_csv_path}")


In [ ]:
import pandas as pd
import unicodedata

# Función para normalizar texto
def normalize_text(text):
    if pd.isna(text):
        return ''
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar acentos
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # Quitar espacios al principio y al final
    return text.strip()

# Rutas a los archivos
base_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/idolina_listas.csv'
secondary_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N.csv'
output_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/idolina_join.csv'
no_match_csv_path = '/Users/ikioriy/Desktop/Idolina_gaona/sin_coincidencias.csv'

# Cargar los archivos CSV, especificando los tipos de datos
df_base = pd.read_csv(base_csv_path, dtype={'grado': int, 'chat_id1': str})
df_secondary = pd.read_csv(secondary_csv_path, dtype={'grado': int, 'chat_id1': str})

# Asegurar que la columna 'curp' exista en ambos DataFrames
if 'curp' not in df_base.columns:
    df_base['curp'] = ''
if 'curp' not in df_secondary.columns:
    df_secondary['curp'] = ''

# Renombrar la columna 'curp' en el df_secondary para evitar conflictos
df_secondary.rename(columns={'curp': 'curp_secondary'}, inplace=True)

# Concatenar las columnas 'apellido' y 'nombre' en df_secondary para que coincidan con la columna 'nombre' en df_base
df_secondary['nombre_completo'] = df_secondary['apellido'] + ' ' + df_secondary['nombre']

# Normalizar las columnas nombre y curp en ambos DataFrames
df_base['nombre_completo'] = df_base['nombre_completo'].apply(normalize_text)
df_secondary['nombre_completo'] = df_secondary['nombre_completo'].apply(normalize_text)
df_base['curp'] = df_base['curp'].apply(normalize_text)
df_secondary['curp_secondary'] = df_secondary['curp_secondary'].apply(normalize_text)

# Realizar el primer merge utilizando 'nombre_completo'
merged_nombre = pd.merge(df_base, df_secondary, on='nombre_completo', how='left', indicator=True)

# Filtrar coincidencias y no coincidencias con nombre completo
df_matched_nombre = merged_nombre[merged_nombre['_merge'] == 'both'].drop(columns=['_merge'])
df_no_match_nombre = merged_nombre[merged_nombre['_merge'] == 'left_only'].drop(columns=['_merge'])

# Realizar el segundo merge con los registros que no coincidieron con el nombre, utilizando la CURP
merged_curp = pd.merge(df_no_match_nombre, df_secondary, left_on='curp', right_on='curp_secondary', how='left', indicator=True)

# Filtrar coincidencias y no coincidencias con la CURP
df_matched_curp = merged_curp[merged_curp['_merge'] == 'both'].drop(columns=['_merge'])
df_no_match_curp = merged_curp[merged_curp['_merge'] == 'left_only'].drop(columns=['_merge'])

# Combinar las coincidencias de ambos merges (nombre completo y CURP)
df_matched = pd.concat([df_matched_nombre, df_matched_curp]).drop_duplicates(subset=['nombre_completo', 'curp'], keep='first')

# Guardar los resultados en archivos CSV
df_matched.to_csv(output_csv_path, index=False)
df_no_match_curp.to_csv(no_match_csv_path, index=False)

print(f"Archivo CSV actualizado guardado en: {output_csv_path}")
print(f"Archivo sin coincidencias guardado en: {no_match_csv_path}")


### ACTUALIZAR EL ARCHIVO MAESTRO CON EL ESTATUS CONFIRMACION

In [ ]:
import pandas as pd
import json

def confirmacion_estatus(clave_instituto: str):
    # Rutas a los archivos
    base_csv_path = f'/Users/ikioriy/Desktop/Exportaciones csv/{clave_instituto}.csv'
    json_path = '/Users/ikioriy/MEGA/Proyectos/telegramregistroLazaro-master/tzolkin-post/bin/Release/net6.0-windows/confirmations.json'
    output_csv_path = f'/Users/ikioriy/Desktop/confirmaciones/{clave_instituto}_confirmaciones.csv'#'/Users/ikioriy/Desktop/las_americas/lazaro_cardenas_confirmacion_general.csv'#'/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_confirmaciones_estatus_actualizado.csv'

    # Cargar el archivo CSV base, especificando los tipos de datos
    df_base = pd.read_csv(base_csv_path, dtype={'chat_id1': str})

    # Cargar el archivo JSON de confirmaciones
    with open(json_path, 'r') as file:
        confirmations = json.load(file)

    # Convertir el archivo JSON a un DataFrame
    df_confirmations = pd.DataFrame(confirmations)

    # Asegurarse de que el UserId en el JSON es tratado como string
    df_confirmations['UserId'] = df_confirmations['UserId'].astype(str)

    # Convertir 'ConfirmedAt' a formato datetime y formatearlo a 'dd/mm/yyyy hh:mm'
    df_confirmations['ConfirmedAt'] = df_confirmations['ConfirmedAt'].apply(
        lambda x: pd.to_datetime(x).strftime('%d/%m/%Y %H:%M')
    )

    # Marcar como "confirmado" en la columna 'confirmacion' donde `chat_id1` coincide con `UserId`
    df_base['confirmacion'] = df_base['chat_id1'].isin(df_confirmations['UserId']).replace({True: 'confirmado', False: ''})

    # Agregar la columna 'fecha confirmacion' con las fechas de confirmación correspondientes
    df_base = df_base.merge(df_confirmations[['UserId', 'ConfirmedAt']], left_on='chat_id1', right_on='UserId', how='left')

    # Renombrar la columna 'ConfirmedAt' a 'fecha confirmacion'
    df_base.rename(columns={'ConfirmedAt': 'fecha confirmacion'}, inplace=True)

    # Eliminar la columna 'UserId' que fue añadida por el merge
    df_base.drop(columns=['UserId'], inplace=True)

    # Guardar el archivo actualizado, manteniendo los tipos de datos
    df_base.to_csv(output_csv_path, index=False)

    print(f"Archivo actualizado guardado en: {output_csv_path}")


In [ ]:
import pandas as pd
import unidecode

# Ruta al archivo JSON de exportación
export_json_path = '/Users/ikioriy/MEGA/Proyectos/telegramregistroLazaro-master/tzolkin-post/bin/Release/net6.0-windows/Exportaciones/14DPR2576Y.json'

# Cargar el archivo JSON de exportación
df_export = pd.read_json(export_json_path)

# Normalización de los nombres y eliminación de duplicados
def normalize_name(name):
    return unidecode.unidecode(name.strip().lower())

# Crear columnas normalizadas de los nombres de autorizados para la eliminación de duplicados
for i in range(1, 5):
    df_export[f'nombre_autorizado_{i}_normalizado'] = df_export[f'nombre_autorizado_{i}'].apply(lambda x: normalize_name(x) if pd.notnull(x) else '')

# Combinar autorizados en una única lista por registro y eliminar duplicados, conservando un máximo de 4
def obtener_autorizados_unicos(row):
    autorizados = []
    for i in range(1, 5):
        nombre_normalizado = row[f'nombre_autorizado_{i}_normalizado']
        telefono = row[f'telefono_autorizado_{i}']
        chat_id = row[f'chat_id{i}']
        if nombre_normalizado and (nombre_normalizado, telefono, chat_id) not in autorizados:
            autorizados.append((nombre_normalizado, telefono, chat_id))
    return autorizados[:4]  # Conservar solo los primeros 4 autorizados únicos

df_export['autorizados_unicos'] = df_export.apply(obtener_autorizados_unicos, axis=1)

# Separar la lista de autorizados únicos en columnas nuevamente
for i in range(1, 5):
    df_export[f'nombre_autorizado_{i}'] = df_export['autorizados_unicos'].apply(lambda x: x[i-1][0] if len(x) >= i else '')
    df_export[f'telefono_autorizado_{i}'] = df_export['autorizados_unicos'].apply(lambda x: x[i-1][1] if len(x) >= i else '')
    df_export[f'chat_id{i}'] = df_export['autorizados_unicos'].apply(lambda x: x[i-1][2] if len(x) >= i else '')

# Eliminar las columnas después de chat_id4
col_index = df_export.columns.get_loc('chat_id4')
df_export = df_export.iloc[:, :col_index+1]

# Guardar el DataFrame actualizado
output_path = '/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_exportacion_limpia.csv'
df_export.to_csv(output_path, index=False)

print(f"Archivo exportado a: {output_path}")


In [ ]:
import pandas as pd
import unidecode

# Rutas a los archivos
base_path = '/Users/ikioriy/Downloads/lazaro_cardenas_base.csv'  # Cambia esto por la ruta correcta del archivo base
export_cleaned_path = '/Users/ikioriy/Desktop/lazaro_cardenas/lazaro_cardenas_exportacion_limpia.csv'

# Cargar los archivos CSV
df_base = pd.read_csv(base_path)
df_export_cleaned = pd.read_csv(export_cleaned_path)

# Normalizar nombres en ambos DataFrames para comparaciones
def normalize_name(name):
    return unidecode.unidecode(name.strip().lower())

df_base['nombre_normalizado'] = df_base['nombre'].apply(lambda x: normalize_name(x) if pd.notnull(x) else '')
df_export_cleaned['nombre_normalizado'] = df_export_cleaned['nombre'].apply(lambda x: normalize_name(x) if pd.notnull(x) else '')

# Filtrar los registros incompletos en el archivo base (donde solo hay datos en la columna nombre)
df_incompletos = df_base[(df_base['nombre'].notnull()) & (df_base.drop(columns=['nombre', 'nombre_normalizado']).isnull().all(axis=1))]

# Encontrar coincidencias y actualizar
for idx, row in df_incompletos.iterrows():
    nombre = row['nombre_normalizado']
    match = df_export_cleaned[df_export_cleaned['nombre_normalizado'] == nombre]
    if not match.empty:
        df_base.loc[idx, :] = match.iloc[0]
        df_base.loc[idx, 'estatus'] = 'ACTUALIZADO'

# Agregar columna estatus si no existe y poner "ACTUALIZADO" en los registros modificados
if 'estatus' not in df_base.columns:
    df_base['estatus'] = 'NO ACTUALIZADO'  # Inicialmente marcamos todos como no actualizados

# Ahora, insertar los registros nuevos del archivo de exportación al archivo base
# Identificar los registros del archivo de exportación que no existen en el archivo base
nuevos_registros = df_export_cleaned[~df_export_cleaned['nombre_normalizado'].isin(df_base['nombre_normalizado'])]

# Marcar los nuevos registros como "NUEVO" en la columna 'estatus'
nuevos_registros['estatus'] = 'NUEVO'

# Concatenar los nuevos registros con el archivo base
df_final = pd.concat([df_base, nuevos_registros], ignore_index=True)

# Eliminar la columna 'nombre_normalizado' que solo se usó para comparaciones
df_final.drop(columns=['nombre_normalizado'], inplace=True)

# Guardar el archivo actualizado
df_final.to_csv(base_path, index=False)

print(f"Archivo base actualizado y guardado en: {base_path}")


In [ ]:
import pandas as pd

# Ruta al archivo CSV
csv_path = '/Users/ikioriy/Desktop/Albert Einstein/autorizados_registros.csv'

# Cargar el archivo CSV
df = pd.read_csv(csv_path)

# Eliminar registros donde la columna 'nombre' esté vacía, contenga valores nulos o solo espacios
df_cleaned = df[df['nombre'].notna() & (df['nombre'].str.strip() != '')].copy()

# Sustituir las celdas vacías en la columna 'chat_id' por 0 usando .loc para evitar advertencias
df_cleaned.loc[:, 'chat_id'] = df_cleaned['chat_id'].fillna(0)

# Eliminar registros duplicados basados en la columna 'id', manteniendo el primer registro
df_cleaned = df_cleaned.drop_duplicates(subset=['id'], keep='first')

# Guardar el DataFrame actualizado en un nuevo archivo CSV
output_csv_path = '/Users/ikioriy/Desktop/Albert Einstein/autorizados_registros_cleaned.csv'
df_cleaned.to_csv(output_csv_path, index=False)

print("Proceso completado y archivo guardado en", output_csv_path)

In [ ]:
import pandas as pd
import unicodedata

# Función para normalizar la columna nombre_completo
def normalizar_nombre(nombre):
    # Eliminar acentos y caracteres especiales
    nombre = ''.join(
        c for c in unicodedata.normalize('NFKD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    # Convertir a minúsculas y eliminar espacios adicionales
    nombre = nombre.lower().strip()
    return nombre

# Rutas de los archivos
archivo_sin_matricula = "/Users/ikioriy/Desktop/Secundaria_153/registros_sin_matricula.csv"
archivo_con_matricula = "/Users/ikioriy/Desktop/Secundaria_153/registros_con_matricula.csv"
archivo_union = "/Users/ikioriy/Desktop/Secundaria_153/registros_union.csv"

# Cargar los archivos en dataframes
df_sin_matricula = pd.read_csv(archivo_sin_matricula)
df_con_matricula = pd.read_csv(archivo_con_matricula)

# Normalizar la columna 'nombre_completo' en ambos dataframes
df_sin_matricula['nombre_completo'] = df_sin_matricula['nombre_completo'].apply(normalizar_nombre)
df_con_matricula['nombre_completo'] = df_con_matricula['nombre_completo'].apply(normalizar_nombre)

# Asegurar que 'matricula' no tenga valores decimales y convertirla en cadena
df_con_matricula['matricula'] = df_con_matricula['matricula'].astype(str).apply(lambda x: x.split('.')[0])

# Realizar la unión basada en la columna 'nombre_completo'
df_union = pd.merge(df_sin_matricula, 
                    df_con_matricula[['nombre_completo', 'curp', 'matricula']], 
                    on='nombre_completo', 
                    how='left')

# Guardar el resultado en un nuevo archivo CSV
df_union.to_csv(archivo_union, index=False)

print(f"Archivo combinado guardado en: {archivo_union}")


In [ ]:
import pandas as pd
import unicodedata

# Función para normalizar la columna nombre_completo
def normalizar_nombre(nombre):
    # Eliminar acentos y caracteres especiales
    nombre = ''.join(
        c for c in unicodedata.normalize('NFKD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    # Convertir a minúsculas y eliminar espacios adicionales
    nombre = nombre.lower().strip()
    return nombre

# Rutas de los archivos
archivo_sin_matricula = "/Users/ikioriy/Desktop/Secundaria_153/registros_sin_matricula.csv"
archivo_con_matricula = "/Users/ikioriy/Desktop/Secundaria_153/registros_con_matricula.csv"
archivo_union = "/Users/ikioriy/Desktop/Secundaria_153/registros_union.csv"
archivo_sin_coincidencias = "/Users/ikioriy/Desktop/Secundaria_153/registros_sin_coincidencias.csv"

# Cargar los archivos en dataframes
df_sin_matricula = pd.read_csv(archivo_sin_matricula)
df_con_matricula = pd.read_csv(archivo_con_matricula)

# Normalizar la columna 'nombre_completo' en ambos dataframes
df_sin_matricula['nombre_completo'] = df_sin_matricula['nombre_completo'].apply(normalizar_nombre)
df_con_matricula['nombre_completo'] = df_con_matricula['nombre_completo'].apply(normalizar_nombre)

# Asegurar que 'matricula' no tenga valores decimales y convertirla en cadena
df_con_matricula['matricula'] = df_con_matricula['matricula'].astype(str).apply(lambda x: x.split('.')[0])

# Realizar la unión basada en la columna 'nombre_completo'
df_union = pd.merge(df_sin_matricula, 
                    df_con_matricula[['nombre_completo', 'curp', 'matricula']], 
                    on='nombre_completo', 
                    how='left')

# Identificar registros sin coincidencias
df_sin_coincidencias = df_union[df_union['curp'].isna() & df_union['matricula'].isna()]

# Guardar el resultado de la unión en un archivo CSV
df_union.to_csv(archivo_union, index=False)
print(f"Archivo combinado guardado en: {archivo_union}")

# Guardar los registros sin coincidencias en otro archivo CSV
df_sin_coincidencias.to_csv(archivo_sin_coincidencias, index=False)
print(f"Archivo sin coincidencias guardado en: {archivo_sin_coincidencias}")


#### SEPARAR DOCUMENTOS DE FORMA AUTOMATIZADA POR GRADO Y GRUPO

In [ ]:
import pandas as pd
import os

# Ruta al archivo original
csv_path = '/Users/ikioriy/Desktop/secundaria_153/secundaria_153_primera_entrega.csv'

# Ruta para guardar los archivos filtrados
output_directory = '/Users/ikioriy/Desktop/secundaria_153/'

# Cargar el archivo CSV
df = pd.read_csv(csv_path)

# Obtener valores únicos de las columnas 'grado' y 'grupo'
grados_unicos = df['grado'].unique()
grupos_unicos = df['grupo'].unique()

# Iterar sobre cada combinación de grado y grupo
for grado in grados_unicos:
    for grupo in grupos_unicos:
        # Filtrar el DataFrame basado en el grado y grupo actuales
        df_filtrado = df[(df['grado'] == grado) & (df['grupo'] == grupo)]
        
        # Si el DataFrame filtrado no está vacío, proceder a guardar el archivo
        if not df_filtrado.empty:
            # Crear el nombre del archivo basado en grado y grupo
            output_file_name = f'secundaria_153_grado_{grado}_grupo_{grupo}.csv'
            output_file_path = os.path.join(output_directory, output_file_name)
            
            # Guardar el DataFrame filtrado en un nuevo archivo CSV
            df_filtrado.to_csv(output_file_path, index=False)
            
            print(f"Archivo filtrado guardado en: {output_file_path}")


In [ ]:
import pandas as pd

# Cargar los archivos CSV
archivo_base = pd.read_csv('/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N.csv')
archivo_secundario = pd.read_csv('/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N_Coincidencias.csv')

# Comparar los archivos y obtener los registros que no están en el archivo secundario usando la columna 'id'
registros_no_encontrados = archivo_base[~archivo_base['id'].isin(archivo_secundario['id'])]

# Guardar los registros no encontrados en un nuevo archivo CSV
registros_no_encontrados.to_csv('/Users/ikioriy/Desktop/Idolina_gaona/Registros_no_encontrados.csv', index=False)

print("Archivo generado con los registros que no están en el archivo secundario.")


### COMBINAR LOS DOCUMENTOS DE LA EXPORTACION DEL BOT CON LOS DATOS EN GOOGLE SHEETS

In [ ]:
import pandas as pd

# Cargar los archivos CSV
archivo_origen = '/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N.csv'
archivo_base = '/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N_base.csv'

df_origen = pd.read_csv(archivo_origen)
df_base = pd.read_csv(archivo_base)

# Realizar el merge de ambos archivos basado en la columna "id"
df_combinado = pd.merge(df_base, df_origen[['id', 'alergias', 'enfermedades cronicas']], on='id', how='left')

# Exportar el resultado a un nuevo archivo CSV
df_combinado.to_csv('/Users/ikioriy/Desktop/Idolina_gaona/14EPR1364N_combinado.csv', index=False)

print("La combinación ha sido completada y guardada en '14EPR1364N_combinado.csv'.")


### ARCHIVO VA EXTRAER LOS DATOS DEL CODIGO QR

In [ ]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import os
import pandas as pd
from PyPDF2 import PdfReader
import re

# Función para decodificar QR de una imagen
def decode_qr_from_image(image_path):
    image = cv2.imread(image_path)
    qr_codes = decode(image)
    for qr in qr_codes:
        qr_data = qr.data.decode('utf-8')
        return qr_data
    return None

# Extraer imágenes de la carpeta y ordenarlas ascendentemente
grado = "2"
image_folder = f"/Users/ikioriy/MEGA/Credenciales/Escuela/Primaria/Valentin Gomez Farias/pdf 2024/{grado}/"
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])

# Inicializar DataFrame para almacenar datos
data = []

# Recorrer las imágenes y extraer los datos del QR
for i, image_file in enumerate(image_files):
    qr_data = decode_qr_from_image(image_file)
    if qr_data:
        data.append({
            "QR": qr_data
        })

# Crear el archivo CSV
df = pd.DataFrame(data)
csv_path = f"/Users/ikioriy/MEGA/Credenciales/Escuela/Primaria/Valentin Gomez Farias/pdf 2024/{grado}_datos_QR2.csv"
df.to_csv(csv_path, index=False)

print(f"Datos extraídos y guardados en: {csv_path}")


### FUSIONAR LOS ARCHIVOS DE LAS LISTAS CON EL FORMLARIO DE JOSEFA ORTIZ DE DOMIGUEZ 

In [ ]:
import pandas as pd
import unidecode

# Función para normalizar el nombre
def normalizar_nombre(nombre):
    nombre = nombre.lower()  # Convertir a minúsculas
    nombre = unidecode.unidecode(nombre)  # Eliminar tildes y caracteres especiales
    nombre = nombre.strip()  # Eliminar espacios en blanco al inicio y al final
    nombre = ' '.join(nombre.split())  # Eliminar espacios múltiples
    return nombre

# Cargar los archivos CSV
archivo1_path = '/Users/ikioriy/MEGA/Credenciales/Escuela/Primaria/Valentin Gomez Farias/pdf 2024/1.csv'
archivo2_path = '/Users/ikioriy/MEGA/Credenciales/Escuela/Primaria/Valentin Gomez Farias/pdf 2024/2.csv'

df1 = pd.read_csv(archivo1_path)
df2 = pd.read_csv(archivo2_path)

# Eliminar registros duplicados en df1 basado en la columna 'nombre_completo'
#df1.drop_duplicates(subset='nombre_completo', inplace=True)

# Normalizar la columna 'nombre_completo' en ambos archivos
df1['nombre_completo_normalizado'] = df1['nombre_completo'].apply(normalizar_nombre)
df2['nombre_completo_normalizado'] = df2['nombre_completo'].apply(normalizar_nombre)

# Eliminar registros duplicados en df1 basado en la columna 'nombre_completo'
df1.drop_duplicates(subset='nombre_completo_normalizado', inplace=True)

# Fusionar los archivos en base al nombre normalizado
df_merged = pd.merge(df1, df2[['nombre_completo_normalizado', 'matricula', 'grado', 'grupo','codigoQr']],
                     on='nombre_completo_normalizado', how='left')

# Buscar los registros del archivo 2 que no tienen coincidencias con el archivo 1
df_sin_coincidencias = df2[~df2['nombre_completo_normalizado'].isin(df_merged['nombre_completo_normalizado'])]

# Eliminar la columna de nombre normalizado
df_merged.drop(columns=['nombre_completo_normalizado'], inplace=True)
df_sin_coincidencias.drop(columns=['nombre_completo_normalizado'], inplace=True)

# Concatenar las coincidencias con los registros sin coincidencias al final
df_final = pd.concat([df_merged, df_sin_coincidencias], ignore_index=True)

# Guardar el archivo final
output_path = '/Users/ikioriy/MEGA/Credenciales/Escuela/Primaria/Valentin Gomez Farias/pdf 2024/archivo_final.csv'
df_final.to_csv(output_path, index=False)

print(f"Archivo final guardado en: {output_path}")


### HOJA DE CONTROL PARA SESIONES FOTOGRAFICAS

In [ ]:
from PIL import Image
from fpdf import FPDF
import os
import pandas as pd

# Ruta de imágenes y CSV
grupo = '1A'
ruta_imagenes = f'/Users/ikioriy/Downloads/Clavijro sesion rafa/{grupo}'
ruta_csv = f'/Users/ikioriy/Downloads/Clavijero hojas de control secundaria/{grupo}.csv'

# Leer archivo CSV
df = pd.read_csv(ruta_csv)

# Crear PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Obtener lista de archivos de imágenes
imagenes = sorted([img for img in os.listdir(ruta_imagenes) if img.endswith(('jpg', 'JPG', 'jpeg', 'png'))])

# Definir tamaño de las imágenes y posición inicial
max_width, max_height = 35, 35  # Tamaño máximo de cada imagen
x_start, y_start = 10, 30  # Posición inicial en la página

# Definir tamaño de la cuadrícula
columns = 4  # 4 imágenes por fila
rows = 4     # 5 filas
image_count = 0

# Función para agregar una página de imágenes
def add_image_page(images, start_index):
    pdf.add_page()
    pdf.set_font('Arial', '', 18)
    pdf.cell(200, 10, f'Hoja de Control de Imágenes - {grupo}', ln=True, align='C')

    x, y = x_start, y_start  # Posición inicial
    for index, img in enumerate(images):
        img_path = os.path.join(ruta_imagenes, img)
        image = Image.open(img_path)
        image.thumbnail((max_width, max_height))
        
        pdf.image(img_path, x=x, y=y, w=max_width)
        
        # Añadir número de imagen debajo de cada una
        pdf.set_xy(x, y + max_height + 2)
        pdf.cell(max_width, 10, f"{start_index + index + 1}", ln=0, align='C')
        
        # Mover la posición horizontal (de izquierda a derecha)
        x += max_width + 10  # Espacio horizontal entre imágenes
        
        if (index + 1) % columns == 0:  # Si se completan 4 columnas, mover a la siguiente fila
            x = x_start
            y += max_height + 15  # Espacio vertical entre filas

# Dividir las imágenes en grupos de 20 (5x4)
for i in range(0, len(imagenes), columns * rows):
    add_image_page(imagenes[i:i + (columns * rows)], i)

# Añadir segunda hoja con nombres de alumnos
pdf.add_page()
pdf.set_font('Arial', '', 10)
pdf.cell(200, 10, 'Hoja de Control - Nombres de Alumnos', ln=True, align='C')
pdf.ln(10)

# Escribir los nombres de los alumnos enumerados
for index, row in df.iterrows():
    nombre_alumno = row['nombre_completo']
    pdf.cell(200, 10, f"{index + 1}. {nombre_alumno}", ln=True)

# Guardar el PDF
output_pdf = f'/Users/ikioriy/Downloads/Clavijero hojas de control secundaria/hoja_control_{grupo}.pdf'
pdf.output(output_pdf)

print(f'PDF guardado en: {output_pdf}')


In [ ]:
from PIL import Image
from fpdf import FPDF
import os
import pandas as pd
import unicodedata

# Función para normalizar nombres
def normalizar_nombre(nombre):
    # Eliminar acentos y convertir a minúsculas
    nfkd_form = unicodedata.normalize('NFKD', nombre)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)]).lower()

# Ruta de imágenes y CSV
grupo = '1A'
ruta_imagenes = '/Users/ikioriy/MEGA/Sesiones fotograficas/Colegio Francisco Javier Clavijero'
ruta_csv = f'/Users/ikioriy/Downloads/Clavijero hojas de control secundaria/{grupo}.csv'

# Leer archivo CSV y agregar columna "nombre_completo"
df = pd.read_csv(ruta_csv)
df['nombre_completo'] = df['nombre'] + ' ' + df['apellidos']
df['nombre_normalizado'] = df['nombre_completo'].apply(normalizar_nombre)

# Crear PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Definir tamaño de las imágenes y posición inicial
max_width, max_height = 35, 35  # Tamaño máximo de cada imagen
x_start, y_start = 10, 30  # Posición inicial en la página

# Definir tamaño de la cuadrícula
columns = 4  # 4 imágenes por fila
rows = 4     # 5 filas
image_count = 0

# Función para agregar una página de imágenes
def add_image_page(alumnos, start_index):
    pdf.add_page()
    pdf.set_font('Arial', '', 8)
    pdf.cell(200, 10, f'Hoja de Control de Imágenes - {grupo}', ln=True, align='C')

    x, y = x_start, y_start  # Posición inicial
    for index, alumno in enumerate(alumnos):
        nombre_completo = alumno['nombre_completo']
        nombre_normalizado = alumno['nombre_normalizado']
        img_path = os.path.join(ruta_imagenes, f"{nombre_normalizado}.jpeg")
        
        # Comprobar si existe la imagen
        if os.path.exists(img_path):
            image = Image.open(img_path)
            image.thumbnail((max_width, max_height))
            
            pdf.image(img_path, x=x, y=y, w=max_width)
            
            # Añadir el nombre del alumno debajo de cada imagen
            pdf.set_xy(x, y + max_height + 10)
            pdf.cell(max_width, 10, f"{nombre_completo}", ln=0, align='C')
        
        # Mover la posición horizontal (de izquierda a derecha)
        x += max_width + 10  # Espacio horizontal entre imágenes
        
        if (index + 1) % columns == 0:  # Si se completan 4 columnas, mover a la siguiente fila
            x = x_start
            y += max_height + 20  # Espacio vertical entre filas

# Dividir los alumnos en grupos de 20 (5x4)
for i in range(0, len(df), columns * rows):
    add_image_page(df.iloc[i:i + (columns * rows)].to_dict('records'), i)

# Añadir segunda hoja con nombres de alumnos
'''pdf.add_page()
pdf.set_font('Arial', '', 10)
pdf.cell(200, 10, 'Hoja de Control - Nombres de Alumnos', ln=True, align='C')
pdf.ln(10)'''

# Escribir los nombres de los alumnos enumerados
'''for index, row in df.iterrows():
    nombre_alumno = row['nombre_completo']
    pdf.cell(200, 10, f"{index + 1}. {nombre_alumno}", ln=True)
'''
# Guardar el PDF
output_pdf = f'/Users/ikioriy/Downloads/Clavijero hojas de control secundaria/hoja_control_{grupo}.pdf'
pdf.output(output_pdf)

print(f'PDF guardado en: {output_pdf}')


### COMPRIMIR DOCUMENTOS PDF

In [ ]:
import os
import subprocess

def compress_pdf(input_path, output_path, power=3):
    """
    Compress PDF using Ghostscript command line interface
    :param input_path: Path to input PDF file
    :param output_path: Path to output PDF file
    :param power: Compression level (0-4), where 0 is lowest and 4 is highest
    """
    quality = {
        0: '/default',
        1: '/prepress',
        2: '/printer',
        3: '/ebook',
        4: '/screen'
    }

    # Ensure Ghostscript command is available
    if not subprocess.call(['gs', '--version'], stdout=subprocess.DEVNULL) == 0:
        raise RuntimeError('Ghostscript not found. Please install it and make sure it\'s in your PATH.')

    try:
        subprocess.call(['gs', '-sDEVICE=pdfwrite', '-dCompatibilityLevel=1.4',
                         f'-dPDFSETTINGS={quality[power]}',
                         '-dNOPAUSE', '-dQUIET', '-dBATCH',
                         f'-sOutputFile={output_path}',
                         input_path]
                       )
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        raise
    
    print(f"Compression complete. Output saved to {output_path}")
    
    # Calculate and print compression ratio
    input_size = os.path.getsize(input_path)
    output_size = os.path.getsize(output_path)
    ratio = 1 - (output_size / input_size)
    print(f"Compression ratio: {ratio:.2%}")

# Usage example
input_pdf = "/Users/ikioriy/Downloads/clavijero_datos/hoja_control_1A.pdf"
output_pdf = "/Users/ikioriy/Downloads/clavijero_datos/hoja_control_1A_compress_ebook.pdf"
compress_pdf(input_pdf, output_pdf, power=3)




### CONVERTIR LOS ARCHIVOS DE LA EXPORTACION DEL BOT EN JSON

In [ ]:
import os
import pandas as pd
import json

def convertir_json_to_csv():
    # Definir las rutas de las carpetas
    input_folder = '/Users/ikioriy/MEGA/Proyectos/telegramregistroLazaro-master/tzolkin-post/bin/Release/net6.0-windows/Exportaciones'
    output_folder = '/Users/ikioriy/Desktop/Exportaciones csv'

    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Recorrer todos los archivos en la carpeta de entrada
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            json_file_path = os.path.join(input_folder, filename)

            # Cargar el archivo JSON
            with open(json_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Convertir a DataFrame de pandas
            df = pd.json_normalize(data)

            # Definir el nombre del archivo CSV de salida
            output_csv_path = os.path.join(output_folder, filename.replace('.json', '.csv'))

            # Guardar el DataFrame como CSV
            df.to_csv(output_csv_path, index=False)

            print(f'Archivo convertido: {output_csv_path}')

    print('Conversión completada.')


### CONVERTIR LOS ARCHVIVOS DE LA EXPORTACION EN JSON VERSION 2 LA CHIDA

In [ ]:
import os
import pandas as pd
import json

def json_to_csv(input_dir, output_dir, overwrite=False):
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Recorrer todos los archivos en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename.replace('.json', '.csv'))
            
            try:
                # Leer el contenido del archivo JSON
                with open(input_path, 'r') as json_file:
                    json_data = json.load(json_file)
                
                # Si el JSON está vacío, crear un DataFrame vacío
                if not json_data:
                    new_df = pd.DataFrame()
                else:
                    new_df = pd.DataFrame(json_data)
                
                # Verificar si el archivo ya existe y si debemos sobrescribir
                if os.path.exists(output_path) and not overwrite:
                    print(f"El archivo {output_path} ya existe. Actualizando...")
                    # Leer el CSV existente
                    existing_df = pd.read_csv(output_path)
                    
                    # Identificar nuevos registros
                    existing_keys = set(existing_df.to_dict('records'))
                    new_keys = set(new_df.to_dict('records'))
                    records_to_add = list(new_keys - existing_keys)
                    
                    # Añadir solo los nuevos registros
                    if records_to_add:
                        updated_df = pd.concat([existing_df, pd.DataFrame(records_to_add)])
                        updated_df.to_csv(output_path, index=False)
                        print(f"Se añadieron {len(records_to_add)} nuevos registros a {output_path}")
                    else:
                        print(f"No se encontraron nuevos registros para añadir a {output_path}")
                else:
                    # Escribir el DataFrame al CSV
                    new_df.to_csv(output_path, index=False)
                    print(f"Se creó un nuevo archivo: {output_path}")
                
                print(f"Archivo procesado: {filename}")
            
            except json.JSONDecodeError:
                print(f"Error: El archivo {filename} no es un JSON válido. Saltando...")
            except Exception as e:
                print(f"Error al procesar {filename}: {str(e)}")

In [ ]:
input_dir = "/Users/ikioriy/MEGA/Proyectos/telegramregistroLazaro-master/tzolkin-post/bin/Release/net6.0-windows/Exportaciones"
output_dir = "/Users/ikioriy/Desktop/Exportaciones csv"

json_to_csv(input_dir, output_dir, overwrite=False)

In [ ]:
clave = "Jose Gonzalez Torres-14EPR1269J"
confirmacion_estatus(clave)

In [ ]:
import pandas as pd
import unicodedata
import re

def normalize_name(name):
    # Convertir a minúsculas y eliminar acentos
    name = unicodedata.normalize('NFKD', name.lower()).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales y espacios extra
    name = re.sub(r'[^a-z0-9\s]', '', name)
    # Reemplazar espacios múltiples por uno solo
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def remove_duplicates(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path)
    
    # Crear una columna normalizada para los nombres
    df['nombre_normalizado'] = df['nombre'].apply(normalize_name)
    
    # Ordenar el DataFrame por 'nombre_normalizado' y 'chat_id1'
    df_sorted = df.sort_values(['nombre_normalizado', 'chat_id1'], ascending=[True, False])
    
    # Eliminar duplicados, manteniendo el primer registro
    df_cleaned = df_sorted.drop_duplicates(subset='nombre_normalizado', keep='first')
    
    # Ordenar el DataFrame resultante por el índice original
    df_cleaned = df_cleaned.sort_index()
    
    # Eliminar la columna 'nombre_normalizado'
    df_cleaned = df_cleaned.drop(columns=['nombre_normalizado'])
    
    # Guardar el DataFrame limpio de vuelta al archivo CSV
    df_cleaned.to_csv(file_path, index=False)
    
    # Imprimir estadísticas
    num_removed = len(df) - len(df_cleaned)
    print(f"Registros originales: {len(df)}")
    print(f"Registros después de eliminar duplicados: {len(df_cleaned)}")
    print(f"Registros eliminados: {num_removed}")

file_path = "/Users/ikioriy/Desktop/Exportaciones csv/Lazaro Cardenas del Rio-14DPR2576Y.csv"
remove_duplicates(file_path)
print("Proceso completado.")

In [ ]:
import pandas as pd
import unicodedata
import re

def normalize_name(name):
    # Convertir a minúsculas y eliminar acentos
    name = unicodedata.normalize('NFKD', str(name).lower()).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales y espacios extra
    name = re.sub(r'[^a-z0-9\s]', '', name)
    # Reemplazar espacios múltiples por uno solo
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def combine_csv_files(file1_path, file2_path, output_path):
    # Leer los archivos CSV
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)
    
    # Crear columnas normalizadas para los nombres
    df1['curp_normalizado'] = df1['curp'].apply(normalize_name)
    df2['curp_normalizado'] = df2['curp'].apply(normalize_name)
    
    # Identificar las columnas únicas en df2
    unique_columns_df2 = [col for col in df2.columns if col not in df1.columns and col != 'curp_normalizado']
    
    # Combinar los DataFrames basándose en los nombres normalizados
    df_combined = pd.merge(df1, df2[['curp_normalizado'] + unique_columns_df2], 
                           on='curp_normalizado', how='left', suffixes=('', '_2'))
    
    # Eliminar la columna 'nombre_normalizado'
    df_combined = df_combined.drop(columns=['curp_normalizado'])
    
    # Guardar el DataFrame combinado en un nuevo archivo CSV
    df_combined.to_csv(output_path, index=False)
    
    # Imprimir estadísticas
    total_records = len(df_combined)
    matched_records = df_combined[unique_columns_df2[0]].notna().sum() if unique_columns_df2 else 0
    print(f"Total de registros en el archivo combinado: {total_records}")
    print(f"Registros con coincidencias: {matched_records}")
    print(f"Registros sin coincidencias: {total_records - matched_records}")
    print(f"Nuevas columnas añadidas del segundo archivo: {unique_columns_df2}")

if __name__ == "__main__":
    file1_path = "/Users/ikioriy/Desktop/secundaria_153/archivo_base.csv"
    file2_path = "/Users/ikioriy/Desktop/secundaria_153/archivo_a_unir.csv"
    output_path = "/Users/ikioriy/Desktop/secundaria_153/combinado.csv"
    combine_csv_files(file1_path, file2_path, output_path)
    print("Proceso completado.")

### COMBINAR LA COLUMNA PAGADO DONDE LA MATRICULA SEA IGUAL

In [ ]:
import pandas as pd

# Cargar los archivos CSV
archivo1_path = '/Users/ikioriy/Desktop/secundaria_153/primer_archivo.csv'
archivo2_path = '/Users/ikioriy/Desktop/secundaria_153/segundo_archivo.csv'

df1 = pd.read_csv(archivo1_path)
df2 = pd.read_csv(archivo2_path)

# Seleccionar las columnas que queremos combinar del segundo archivo
columnas_a_combinar = ['matricula', 'estatus pago', 'confirmacion 2', 'fecha confirmacion 2']

# Combinar las columnas seleccionadas del segundo archivo al primero usando 'matricula' como referencia
df_combined = pd.merge(df1, df2[columnas_a_combinar], on='matricula', how='left')

# Guardar el archivo combinado
output_path = '/Users/ikioriy/Desktop/secundaria_153/archivo_combined.csv'
df_combined.to_csv(output_path, index=False)

print(f"Archivo combinado guardado en: {output_path}")


### SEPARAR NOMBRE Y APELLIDOS DE NOMBRE_COMPLETO

In [ ]:
import pandas as pd

# Cargar el archivo CSV
file_path = '/Users/ikioriy/Desktop/secundaria_153/archivo_matriculas_separar_nombre.csv'
df = pd.read_csv(file_path)

# Lista de partículas comunes en apellidos compuestos
particulas_apellido = ['de', 'del', 'la', 'las', 'los', 'y', 'san', 'santa']

# Función para capitalizar correctamente los nombres y apellidos
def capitalizar_nombre(nombre):
    return ' '.join([palabra.capitalize() for palabra in nombre.split()])

# Función para identificar si una palabra es una partícula de apellido
def es_particula(palabra):
    return palabra.lower() in particulas_apellido

# Función mejorada para separar apellidos y nombres
def separar_nombre_completo(nombre_completo):
    partes = nombre_completo.split()
    
    apellidos = []
    nombres = []
    i = 0
    
    # Mientras recorremos las palabras, tomamos en cuenta las partículas de apellidos compuestos
    while i < len(partes):
        apellidos.append(partes[i])
        if i + 1 < len(partes) and es_particula(partes[i].lower()):
            i += 1
            apellidos.append(partes[i])
        elif i + 1 < len(partes) and es_particula(partes[i + 1].lower()):  # Mirar si la siguiente palabra es una partícula
            i += 1
            apellidos.append(partes[i])
        elif len(apellidos) >= 2:  # Después de obtener al menos dos apellidos, paramos
            break
        i += 1
    
    # Lo que queda son los nombres
    nombres = partes[i + 1:] if i < len(partes) else []
    
    # Capitalizamos apellidos y nombres
    return capitalizar_nombre(' '.join(apellidos)), capitalizar_nombre(' '.join(nombres))

# Aplicar la función de separación
df[['apellidos', 'nombres']] = df['nombre_completo'].apply(lambda x: pd.Series(separar_nombre_completo(x)))

# Guardar el nuevo archivo CSV
output_file_path = '/Users/ikioriy/Desktop/secundaria_153/archivo_matriculas_separado.csv'
df.to_csv(output_file_path, index=False)

print(f"Archivo guardado en: {output_file_path}")


### DIGITALIZAR LAS LISTAS DESDE JPEG CON OCR

In [ ]:
import os
import pytesseract
from PIL import Image
import csv

# En macOS, generalmente no es necesario especificar la ruta de Tesseract
# pero si lo fuera, descomenta y ajusta la siguiente línea:
# pytesseract.pytesseract.tesseract_cmd = '/usr/local/bin/tesseract'

# Configurar el idioma para español
idioma = 'spa'  # Código para español

def procesar_imagen(ruta_imagen):
    # Extraer grado y grupo del nombre del archivo
    nombre_archivo = os.path.basename(ruta_imagen)
    grado, grupo = nombre_archivo.split('.')[0].split(' ')[-2:]
    
    # Leer la imagen y extraer el texto
    imagen = Image.open(ruta_imagen)
    texto = pytesseract.image_to_string(imagen, lang=idioma)
    
    return procesar_lista(texto, grado, grupo)

def procesar_lista(texto, grado, grupo):
    resultado = []
    for linea in texto.split('\n'):
        partes = linea.strip().split()
        if partes and partes[0].isdigit():
            numero = int(partes[0])
            nombre_completo = ' '.join(partes[1:])
            
            # Separar apellidos y nombres
            palabras = nombre_completo.split()
            apellidos = []
            nombres = []
            for palabra in palabras:
                if palabra.isupper():
                    apellidos.append(palabra.capitalize())
                else:
                    nombres.append(palabra)
            
            apellidos = ' '.join(apellidos)
            nombres = ' '.join(nombres)
            
            # Crear la línea de salida como una lista
            linea_salida = [numero, apellidos, nombres, grado, grupo]
            resultado.append(linea_salida)
    
    return resultado

def procesar_directorio(ruta_directorio):
    resultados = []
    for archivo in os.listdir(ruta_directorio):
        if archivo.lower().endswith(('.jpg', '.jpeg')):
            ruta_completa = os.path.join(ruta_directorio, archivo)
            resultado = procesar_imagen(ruta_completa)
            resultados.extend(resultado)
    
    return resultados

# Ejemplo de uso
ruta_directorio = '/Users//ikioriy/Downloads/Listas martin de coruña'  # Ajusta esta ruta
resultados = procesar_directorio(ruta_directorio)

# Guardar la salida en un archivo CSV
ruta_salida = '/Users//ikioriy/Downloads/Listas martin de coruña/resultados.csv'
with open(ruta_salida, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Número', 'Apellidos', 'Nombres', 'Grado', 'Grupo'])  # Encabezados
    writer.writerows(resultados)

print(f"Se ha generado el archivo {ruta_salida} con los resultados.")